In [1]:
import pandas as pd
import numpy as np
import collections
import re

<h1>For Antibiotics</h1>

In [2]:
medication=pd.read_csv("medication.csv")
# subselect the variable columns
medication_columns = ['drugorderoffset', 'patientunitstayid','drugstartoffset','drugname', 'dosage', 'routeadmin', 'drugstopoffset']
medication_subset = medication[medication_columns]
#medication_subset = medication_subset[medication_subset['routeadmin']== 'IV']
del medication
antibiotics_list=['adoxa','ala-tet','alodox','amikacin','amikin','amoxicillin','clavulanate','ampicillin',
                 'augmentin','avelox','avidoxy','azactam','azithromycin','aztreonam','axetil','bactocill',
                 'bactrim','bethkis','biaxin','bicillin l-a','cayston','cefazolin','cedax','cefoxitin',
                 'ceftazidime','cefaclor','cefadroxil','cefdinir','cefditoren','cefepime','cefotetan','cefotaxime',
                 'cefpodoxime','cefprozil','ceftibuten','ceftin','cefuroxime','cefuroxime','cephalexin','chloramphenicol',
                 'cipro','ciprofloxacin','claforan','clarithromycin','cleocin','clindamycin','cubicin','dicloxacillin',
                 'doryx','doxycycline','duricef','dynacin','ery-tab','eryped','eryc','erythrocin','erythromycin','factive',
                 'flagyl','fortaz','furadantin','garamycin','gentamicin','kanamycin','keflex','ketek','levaquin',
                 'levofloxacin','lincocin','macrobid','macrodantin','maxipime','mefoxin','metronidazole','minocin',
                 'minocycline','monodox','monurol','morgidox','moxatag','moxifloxacin','myrac','nafcillin sodium',
                 'nicazel doxy 30','nitrofurantoin','noroxin','ocudox','ofloxacin','omnicef','oracea','oraxyl','oxacillin',
                 'pc pen vk','pce dispertab','panixine','pediazole','penicillin','periostat','pfizerpen','piperacillin',
                 'tazobactam','primsol','proquin','raniclor','rifadin','rifampin','rocephin','smz-tmp','septra','septra ds',
                 'septra','solodyn','spectracef','streptomycin sulfate','sulfadiazine','sulfamethoxazole','trimethoprim',
                 'sulfatrim','sulfisoxazole','suprax','synercid','tazicef','tetracycline','timentin','tobi','tobramycin',
                 'trimethoprim','unasyn','vancocin','vancomycin','vantin','vibativ','vibra-tabs','vibramycin','zinacef',
                 'zithromax','zmax','zosyn','zyvox']

medication_subset_filt = medication_subset[medication_subset.drugname.str.contains('|'.join(antibiotics_list), flags=re.IGNORECASE, na=False)]
del medication_subset

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
medication_subset_filt.head()

,drugorderoffset,patientunitstayid,drugstartoffset,drugname,dosage,routeadmin,drugstopoffset
25,12031,141194,13422,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),"1,250 3",IV,12622
32,3,141203,-129,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,500 3,IV,1058
41,1097,141203,1281,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,500 3,IV,2414
42,1186,141203,2361,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),"1,250 3",IV,2414
55,2,141227,-1323,CEFEPIME HCL 2 G IJ SOLR,2000 MG,IV,507


<h1>For Body Fluids Culture</h1>

In [4]:
treatment=pd.read_csv("treatment.csv")
treatment_columns = ['treatmentoffset', 'patientunitstayid','treatmentstring']
treatment_subset = treatment[treatment_columns]
treatment_subset = treatment_subset[treatment_subset['treatmentstring'].str.contains("cardiovascular")]
treatment_columns = ['treatmentoffset', 'patientunitstayid']
treatment_subset = treatment_subset[treatment_columns]
treatment_subset_new = treatment_subset.rename(columns={'treatmentoffset': 'culturetakenoffset'})

microlab=pd.read_csv("microLab.csv")
microlab_columns = ['culturetakenoffset', 'patientunitstayid']
microlab_subset = microlab[microlab_columns]
treatment_microlab = pd.concat([treatment_subset_new,microlab_subset])

ABX_BC_pid_offset = pd.merge(medication_subset_filt[['patientunitstayid', 'drugorderoffset']], treatment_microlab[['patientunitstayid', 'culturetakenoffset']], on='patientunitstayid', how='inner').drop_duplicates()
# For each PID, the difference of each blood culture and each antibiotics time is calculated 
ABX_BC_pid_offset.loc[(ABX_BC_pid_offset['drugorderoffset'] < ABX_BC_pid_offset['culturetakenoffset']) & (ABX_BC_pid_offset['culturetakenoffset'] - ABX_BC_pid_offset['drugorderoffset']  <=24*3600), 'tSuspicion'] = pd.DataFrame([ABX_BC_pid_offset['culturetakenoffset'], ABX_BC_pid_offset['drugorderoffset']]).min()
ABX_BC_pid_offset.loc[(ABX_BC_pid_offset['culturetakenoffset'] < ABX_BC_pid_offset['drugorderoffset']) & (ABX_BC_pid_offset['drugorderoffset'] - ABX_BC_pid_offset['culturetakenoffset']  <=72*3600), 'tSuspicion'] = pd.DataFrame([ABX_BC_pid_offset['drugorderoffset'], ABX_BC_pid_offset['culturetakenoffset']]).min()

ABX_BC_pid_offset.drop_duplicates(inplace=True)
ABX_BC_pid_offset_Clean = ABX_BC_pid_offset[np.isfinite(ABX_BC_pid_offset['tSuspicion'])]
ABX_BC_pid_offset_Clean.to_csv("ABX_BC_pid_offset_Clean_Tsuspicion.csv", sep=',', index=False)

suspicion = pd.read_csv("ABX_BC_pid_offset_Clean_Tsuspicion.csv")
suspicionUnique = suspicion[['patientunitstayid', 'tSuspicion']].drop_duplicates()
suspicionUnique.to_csv("ABX_BC_pid_offset_Clean_TsuspicionUnique.csv", sep=',', index=False)
suspicionMax = suspicion.groupby(['patientunitstayid']).agg({'tSuspicion': ['max']})
suspicionMax.columns = suspicionMax.columns.droplevel(0)
suspicionMax.to_csv("ABX_BC_pid_offset_Clean_TsuspicionMax.csv", sep=',')

patientIDs_IV = suspicionUnique.patientunitstayid.unique().tolist()
patientIDs_IV_sub = pd.DataFrame(patientIDs_IV)
patientIDs_IV_sub.columns=['patientunitstayid']
del medication_subset_filt

In [5]:
patientIDs_IV_sub

,patientunitstayid
0,244707
1,244745
2,246013
3,246259
4,253346
5,256461
6,258786
7,258915
8,262382
9,263828


<h1>For GCS, MAP, Ventilator</h1>

In [6]:
nurseChart=pd.read_csv("nurseCharting.csv")
nurseChart = pd.merge(nurseChart,patientIDs_IV_sub, on='patientunitstayid', how='inner').drop_duplicates()
nurseChartGCS = nurseChart[(nurseChart['nursingchartcelltypevallabel']== 'Glasgow coma score')]
#nurseChartGCS_MAP = nurseChart[(nurseChart['nursingchartcelltypevallabel']== 'Glasgow coma score') | (nurseChart['nursingchartcelltypevallabel']== 'MAP (mmHg)')]
nurseChartGCS.to_csv("nurseChartGCS.csv", index=False)
nurseChartMAP = nurseChart[nurseChart['nursingchartcelltypevallabel']== 'MAP (mmHg)']
nurseChartMAP.to_csv("nurseChartMAP.csv", index=False)
nurseChartVent = nurseChart[nurseChart['nursingchartcelltypevallabel']== 'O2 Admin Device']
nurseChartVent.to_csv("nurseChartVent.csv", index=False)
del nurseChart
nurseChartGCS=pd.read_csv("nurseChartGCS.csv")

/sw/acf/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<h1>SOFA GCS</h1>

In [7]:
gcs=[]
d_gcs=[]
for row in nurseChartGCS.itertuples():
    if((row.nursingchartcelltypecat=="Scores" and row.nursingchartcelltypevallabel=="Glasgow coma score")):
        d_gcs.append(row)    
d_gcs=pd.DataFrame(d_gcs)
d_gcs['SOFA_GCS']=""

d_gcs.nursingchartvalue=pd.to_numeric(d_gcs.nursingchartvalue,errors='coerce')
d_gcs=d_gcs.dropna(subset=['nursingchartvalue'])
d_gcs.nursingchartvalue=d_gcs.nursingchartvalue.astype(int)

d_gcs.loc[d_gcs['nursingchartvalue']>=15,['SOFA_GCS']]=0
d_gcs.loc[d_gcs['nursingchartvalue']<15,['SOFA_GCS']]=1
d_gcs.loc[d_gcs['nursingchartvalue']<13,['SOFA_GCS']]=2
d_gcs.loc[d_gcs['nursingchartvalue']<10,['SOFA_GCS']]=3
d_gcs.loc[d_gcs['nursingchartvalue']<6,['SOFA_GCS']]=4
d_gcs.to_csv("d_gcs_041119.csv", index=False)

<h1>For SOFA Cardiovascular</h1>

In [8]:
infusiondrug = pd.read_csv("vasopressor_norm_201903161640.csv")
# subselect the variable columns
columns = ['infusiondrugid','patientunitstayid', 'drug', 'drugrate_norm', 'infusionoffset']
infusiondrugSub = infusiondrug[columns]
del infusiondrug
infusiondrugPSub = pd.merge(infusiondrugSub,patientIDs_IV_sub, on='patientunitstayid', how='inner').drop_duplicates()
# pivot the infusiondrug table to put variables into columns
infusiondrugP = infusiondrugPSub.pivot_table(index=['patientunitstayid', 'infusionoffset'], columns='drug', values='drugrate_norm').reset_index()

nurseChartMAP=pd.read_csv("nurseChartMAP.csv")
SOFA_Cardiovascular = pd.merge(infusiondrugP,nurseChartMAP, on='patientunitstayid', how='outer').drop_duplicates()
SOFA_Cardiovascular.loc[((SOFA_Cardiovascular['dopamine'] >15)) | ((SOFA_Cardiovascular['epinephrine'] >0.1)) | ((SOFA_Cardiovascular['norepinephrine'] >0.1)), 'SOFA_Cardiovascular'] = 4
SOFA_Cardiovascular.loc[((SOFA_Cardiovascular['dopamine'] <15)) | ((SOFA_Cardiovascular['epinephrine'] <=0.1)) | ((SOFA_Cardiovascular['norepinephrine'] <=0.1)), 'SOFA_Cardiovascular'] = 3
SOFA_Cardiovascular.loc[((SOFA_Cardiovascular['dopamine'] <5)) | ((SOFA_Cardiovascular['dobutamine'] >0)), 'SOFA_Cardiovascular'] = 2
SOFA_Cardiovascular.loc[SOFA_Cardiovascular['nursingchartvalue']<70,['SOFA_Cardiovascular']]=1
SOFA_Cardiovascular.loc[SOFA_Cardiovascular['nursingchartvalue']>71,['SOFA_Cardiovascular']]=0

SOFA_Cardiovascular['offset'] = np.where(SOFA_Cardiovascular['infusionoffset']>0, SOFA_Cardiovascular['infusionoffset'], SOFA_Cardiovascular['nursingchartoffset'])
SOFA_Cardiovascular = SOFA_Cardiovascular.dropna(subset = ['offset'])
SOFA_Cardiovascular['offset'] = SOFA_Cardiovascular['offset'].astype('int64')
SOFA_Cardiovascular.to_csv("SOFA_Cardiovascular.csv", index=False)

<h1>For Lab Related SOFA ('creatinine', 'FiO2', 'paO2', 'platelets',
       'bilirubin')</h1>

In [9]:
lab=pd.read_csv("lab_SOFAvariables.csv")
# subselect the variable columns
columns = ['labid','patientunitstayid', 'labresultoffset', 'labname', 'labresult']
labSub = lab[columns]
labSubP = pd.merge(labSub,patientIDs_IV_sub, on='patientunitstayid', how='inner').drop_duplicates()
# pivot the infusiondrug table to put variables into columns
labSubPivot = labSubP.pivot_table(index=['patientunitstayid','labresultoffset'], columns='labname', values='labresult').reset_index()

/sw/acf/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
nurseChartVent=pd.read_csv("nurseChartVent.csv")
#Probably should have not used the offset time in the 'on' condition; as long as patient was on ventilator once its fine ?
labSubPivotVent = pd.merge(labSubPivot,nurseChartVent, left_on=['patientunitstayid','labresultoffset'], right_on=['patientunitstayid','nursingchartoffset'], how='left').drop_duplicates()
labSubPivotVent['offset'] = np.where(labSubPivotVent['nursingchartoffset']>0, labSubPivotVent['nursingchartoffset'], labSubPivotVent['labresultoffset'])

In [11]:
labSubPivotfiltered = labSubPivotVent

labSubPivotfiltered.loc[(labSubPivotfiltered['platelets x 1000'] >=150), 'SOFA_Coagulation'] = 0
labSubPivotfiltered.loc[(labSubPivotfiltered['platelets x 1000'] <150), 'SOFA_Coagulation'] = 1
labSubPivotfiltered.loc[(labSubPivotfiltered['platelets x 1000'] <100) , 'SOFA_Coagulation'] = 2
labSubPivotfiltered.loc[(labSubPivotfiltered['platelets x 1000'] <50), 'SOFA_Coagulation'] = 3
labSubPivotfiltered.loc[(labSubPivotfiltered['platelets x 1000'] <20), 'SOFA_Coagulation'] = 4

labSubPivotfiltered.loc[(labSubPivotfiltered['total bilirubin'] <1.2), 'SOFA_Liver'] = 0
labSubPivotfiltered.loc[(labSubPivotfiltered['total bilirubin'] >=1.2) & (labSubPivotfiltered['total bilirubin'] <=1.9), 'SOFA_Liver'] = 1
labSubPivotfiltered.loc[(labSubPivotfiltered['total bilirubin'] >=2) & (labSubPivotfiltered['total bilirubin'] <=5.9), 'SOFA_Liver'] = 2
labSubPivotfiltered.loc[(labSubPivotfiltered['total bilirubin'] >=6) & (labSubPivotfiltered['total bilirubin'] <=11.9), 'SOFA_Liver'] = 3
labSubPivotfiltered.loc[(labSubPivotfiltered['total bilirubin'] >12), 'SOFA_Liver'] = 4

labSubPivotfiltered['paO2FiO2_Ratio'] = np.where(labSubPivotfiltered['FiO2']>0, labSubPivotfiltered['paO2']/labSubPivotfiltered['FiO2'], None)
labSubPivotfiltered.loc[(labSubPivotfiltered['paO2FiO2_Ratio'] >=400), 'SOFA_Respiration'] = 0
labSubPivotfiltered.loc[(labSubPivotfiltered['paO2FiO2_Ratio'] <400), 'SOFA_Respiration'] = 1
labSubPivotfiltered.loc[(labSubPivotfiltered['paO2FiO2_Ratio'] <300), 'SOFA_Respiration'] = 2
labSubPivotfiltered.loc[((labSubPivotfiltered['paO2FiO2_Ratio'] <200) & (labSubPivotfiltered['nursingchartvalue'] =='ventilator')), 'SOFA_Respiration'] = 3
labSubPivotfiltered.loc[((labSubPivotfiltered['paO2FiO2_Ratio'] <100) & (labSubPivotfiltered['nursingchartvalue'] =='ventilator')), 'SOFA_Respiration'] = 4

labSubPivotfiltered['SOFA_Renal'] = 0
labSubPivotfiltered.loc[((labSubPivotfiltered['creatinine'] >=1.2) & (labSubPivotfiltered['creatinine'] <=1.9)), 'SOFA_Renal'] = 1
labSubPivotfiltered.loc[((labSubPivotfiltered['creatinine'] >=2) & (labSubPivotfiltered['creatinine'] <=3.4)), 'SOFA_Renal'] = 2
labSubPivotfiltered.loc[((labSubPivotfiltered['creatinine'] >=3.5) & (labSubPivotfiltered['creatinine'] <=4.9)) | (labSubPivotfiltered['urinary creatinine'] <200), 'SOFA_Renal'] = 3
labSubPivotfiltered.loc[(labSubPivotfiltered['creatinine'] >5) | (labSubPivotfiltered['urinary creatinine'] <200), 'SOFA_Renal'] = 4

labSubPivotfiltered.to_csv("labSubPivotfiltered_041119.csv", index=False)

In [12]:
GCS_Cardiovascular = pd.merge(d_gcs,SOFA_Cardiovascular, left_on=['patientunitstayid','nursingchartoffset'], right_on=['patientunitstayid','offset'] , how='outer').drop_duplicates()
GCS_Cardiovascular['offset'] = np.where(GCS_Cardiovascular['nursingchartoffset_x']>0, GCS_Cardiovascular['nursingchartoffset_x'], GCS_Cardiovascular['offset'])
finalSOFA = pd.merge(labSubPivotfiltered,GCS_Cardiovascular, left_on=['patientunitstayid','offset'], right_on=['patientunitstayid','offset'], how='outer').drop_duplicates()
finalSOFA.to_csv("finalSOFA.csv", index=False)
finalSOFA_filter = finalSOFA[['patientunitstayid','offset','SOFA_Coagulation','SOFA_Liver','SOFA_Respiration','SOFA_Renal','SOFA_GCS','SOFA_Cardiovascular']]
finalSOFA_filter.to_csv("finalSOFA_filter.csv", index=False)

<h1>Overall SOFA</h1>

In [13]:
finalSOFA_Total=pd.read_csv("finalSOFA_filter.csv")
finalSOFA_Total.replace(np.nan, 0, inplace=True)
finalSOFA_Total['SOFA_value'] = finalSOFA_Total["SOFA_Coagulation"] + finalSOFA_Total["SOFA_Liver"] + finalSOFA_Total['SOFA_Respiration'] + finalSOFA_Total["SOFA_Cardiovascular"] + finalSOFA_Total["SOFA_Renal"] + finalSOFA_Total["SOFA_GCS"]
finalSOFAgroupby = finalSOFA_Total.groupby(['patientunitstayid']).nunique()
finalSOFAgroupby.to_csv("finalSOFA_groupby_patients.csv", index=False)
finalSOFA_Total = finalSOFA_Total.dropna(subset = ['offset'])
finalSOFA_Total['offset'] = finalSOFA_Total['offset'].astype('int64')
finalSOFA_Total = finalSOFA_Total.sort_values(['patientunitstayid','offset'],ascending=[1,0])

In [14]:
# Forward fill SOFA_value
finalSOFA_Total['SOFA_value'] = finalSOFA_Total.groupby('patientunitstayid')['SOFA_value'].transform(lambda v: v.ffill())
# This is just calculating consecutive difference of SOFA_values , store in dSOFA variable
finalSOFA_Total['dSOFA'] = finalSOFA_Total.groupby(['patientunitstayid'])['SOFA_value'].transform(lambda x: x.diff()).fillna(0)
finalSOFA_Total['diffs'] = finalSOFA_Total.groupby(['patientunitstayid'])['offset'].transform(lambda x: x.diff()).fillna(0)
finalSOFA_Total['cumulative_seconds'] = finalSOFA_Total.groupby(['patientunitstayid'])['diffs'].apply(lambda x: x.cumsum())
finalSOFA_Total['offset_1'] = finalSOFA_Total['offset']
finalSOFA_Total['offset_1'] = finalSOFA_Total['offset_1'].astype('int64')
finalSOFA_Total['datetime_colum'] = pd.to_datetime(finalSOFA_Total['offset'])

<h1>Rolling 24-hour SOFA</h1>

In [ ]:
finalSOFA_Total['diffs'] = finalSOFA_Total.groupby(['patientunitstayid'])['offset'].transform(lambda x: -x.diff()).fillna(0)
finalSOFA_Total['cumulative_seconds'] = finalSOFA_Total.groupby(['patientunitstayid'])['diffs'].apply(lambda x: x.cumsum())
X0 = finalSOFA_Total
maxNumOfHours = 72
X0 = X0[(X0["cumulative_seconds"] >= (maxNumOfHours * 60))]

X0['SOFA_val'] = pd.to_numeric(X0['SOFA_value'])

df_agg1 = (X0.join(X0.sort_values("datetime_colum")
                   .set_index('datetime_colum')
                   .groupby("patientunitstayid")
                   .rolling("1d")["SOFA_val"].max()
                   .rename('aggval'), on=['patientunitstayid','datetime_colum'])
                   .sort_values(["patientunitstayid", "datetime_colum"]))

df_agg1['diff'] = df_agg1.groupby(['patientunitstayid'])['aggval'].diff().fillna(0)
df_agg1.to_csv("eICU_Sepsis_SOFA_24hrWin.csv", index=False)

/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


<h1>tSuspicion, tSepsis, tSOFA</h1>

In [16]:
df_agg1 = pd.read_csv("eICU_Sepsis_SOFA_24hrWin.csv")
suspicionMax = pd.read_csv("ABX_BC_pid_offset_Clean_TsuspicionMax.csv")
dfMini = pd.merge(df_agg1, suspicionMax, how='inner', left_on=['patientunitstayid'], right_on=['patientunitstayid'] ).drop_duplicates()

#variable max is the tSuspicion max time
X1 = dfMini
maxNumOfHours = 12
X1 = X1[(X1['max'] + maxNumOfHours*60 >= X1['offset'])]

dfMini=X1
dfMini = dfMini.where(dfMini.notnull(), None)
dfMini = dfMini.groupby(['patientunitstayid'], as_index=False).fillna(method='ffill') 
dfMini = dfMini.groupby(['patientunitstayid'], as_index=False).fillna(method='bfill')
dfMini['tSus_ts'] = np.where( (dfMini['max']) < 0, None, (dfMini['max']))
dfMini['observe'] = np.where( ((dfMini.offset >= (dfMini.tSus_ts - 60*24 )) & (dfMini.offset <= (dfMini.tSus_ts + 60*12 ))), 1 , 0)
dfMini['OF_flag'] = np.where( ((dfMini['diff'] > 1) & (dfMini['observe'] == 1)), 1, 0 )

OF_Flag = dfMini[(dfMini.OF_flag == 1)]['patientunitstayid'].drop_duplicates()
OF_Flag.to_csv("OF_Flag.csv", index=False)

dfMini['tOnset'] = np.where( (dfMini['OF_flag'] == 1), dfMini['offset'], None  )
dfMini = dfMini.groupby(['patientunitstayid'], as_index=False).fillna(method='ffill') 
dfMini = dfMini.groupby(['patientunitstayid'], as_index=False).fillna(method='bfill')
dfMini['sepsisClass'] = np.where( ( dfMini['OF_flag'] == 1 ), 1, None )
dfMini = dfMini.groupby(['patientunitstayid'], as_index=False).fillna(method='ffill') 
dfMini = dfMini.groupby(['patientunitstayid'], as_index=False).fillna(method='bfill')
dfMini.to_csv("SOFA_SepsisClass.csv", index=False)

dfPIDtSuspicion = dfMini[(dfMini['sepsisClass'] == 1)][['patientunitstayid','tSus_ts']].drop_duplicates()
dfPID = dfMini[(dfMini['sepsisClass'] == 1)]['patientunitstayid'].drop_duplicates()
dfPIDtSuspicion.to_csv("SOFA_SepsisClass_patientunitstayid_tSusspicion.csv", index=False)
dfPID.to_csv("SOFA_SepsisClass_patientunitstayid.csv", index=False)

dfPIDtOnset = dfMini[(dfMini['sepsisClass'] == 1)][['patientunitstayid','tSus_ts']].drop_duplicates()

dfPIDSepsis = dfMini[['patientunitstayid']].merge(dfPIDtOnset, on='patientunitstayid', how='outer', indicator=True).drop_duplicates()
dfPIDSepsis = dfPIDSepsis[dfPIDSepsis['_merge']=='both'][['patientunitstayid','tSus_ts']]

dfPIDnonSepsis = dfMini[['patientunitstayid']].merge(dfPIDtOnset, on='patientunitstayid', how='outer', indicator=True).drop_duplicates()
dfPIDnonSepsis = dfPIDnonSepsis[dfPIDnonSepsis['_merge']=='left_only'][['patientunitstayid','tSus_ts']]

In [17]:
dfPIDtOnset['AoF_flag']=1
dfPIDtOnset

,patientunitstayid,tSus_ts,AoF_flag
0,242040,188,1
1071,242434,1,1
1112,242544,31,1
1152,242851,466,1
1179,243208,892,1
2308,243629,70,1
2328,243643,1133,1
2512,243765,102,1
3778,243961,407,1
3950,244396,295,1


In [18]:
dfPIDSepsis['AoF_flag']=1

In [19]:
dfPIDnonSepsis['AoF_flag']=0

<h1>Extract Vitals</h1>

In [20]:
vitP = pd.read_csv("vitalPeriodic.csv")
vitP_sepsis = pd.merge(vitP, dfPIDSepsis, on='patientunitstayid', how='inner').drop_duplicates()
vitP_sepsis.to_csv("vitP_sepsis.csv", index=False)
vitP_nonsepsis = pd.merge(vitP, dfPIDnonSepsis, on='patientunitstayid', how='inner').drop_duplicates()
vitP_nonsepsis.to_csv("vitP_nonsepsis.csv", index=False)
vitP_subset = pd.concat([vitP_sepsis,vitP_nonsepsis])
vitP_subset.to_csv("vitP_subset.csv", index=False)

In [21]:
vitP_subset.columns

Index([u'vitalperiodicid', u'patientunitstayid', u'observationoffset',
       u'temperature', u'sao2', u'heartrate', u'respiration', u'cvp', u'etco2',
       u'systemicsystolic', u'systemicdiastolic', u'systemicmean',
       u'pasystolic', u'padiastolic', u'pamean', u'st1', u'st2', u'st3',
       u'icp', u'tSus_ts', u'AoF_flag'],
      dtype='object')

In [22]:
vitP_subset.nunique()

vitalperiodicid      32771255
patientunitstayid       20792
observationoffset      121318
temperature              1641
sao2                      101
heartrate                 272
respiration               197
cvp                       499
etco2                     189
systemicsystolic          429
systemicdiastolic         432
systemicmean              438
pasystolic                399
padiastolic               384
pamean                    430
st1                      1772
st2                      1950
st3                      2056
icp                       425
tSus_ts                  1560
AoF_flag                    2
dtype: int64